In [1]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [2]:
narrative_path = '../data_to_model/keras_data/model_df_full_narrative.csv'
train_path = '../data_to_model/Train and Test Sets/train_df_count.csv'

In [3]:
narrative_df = pd.read_csv(narrative_path)
narrative_df.drop(columns= ['primary_mechanism'], inplace = True )

In [4]:
train_df = pd.read_csv(train_path).merge(narrative_df,how='left', on=['cpsc_case_number'] )
#train_df

In [5]:
from sklearn.preprocessing import StandardScaler

features = train_df[['age']].values
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
#features_scaled


In [6]:
one_hot_tab_features = pd.get_dummies(train_df[['diagnosis', 'sex', 'body_part' ]] )
#print(one_hot_tab_features)
one_hot_tab_features = one_hot_tab_features.values
#print(one_hot_tab_features)

In [7]:
tabular_features_scaled = np.hstack([features_scaled, one_hot_tab_features])
print(tabular_features_scaled)
print(tabular_features_scaled.shape)

[[ 1.62316007  0.          0.         ...  0.          0.
   0.        ]
 [-1.25396467  0.          0.         ...  0.          0.
   0.        ]
 [ 0.39010661  0.          0.         ...  1.          0.
   0.        ]
 ...
 [ 0.39010661  0.          1.         ...  0.          0.
   0.        ]
 [ 0.59561552  0.          0.         ...  0.          0.
   0.        ]
 [ 0.69836998  0.          0.         ...  0.          0.
   0.        ]]
(1597, 36)


In [8]:
text_np = np.array(train_df['narrative'])
train_labels_np = np.array(train_df['primary_mechanism'])
#train_labels_np_bin = np.where(train_labels_np == 'Multi-sport', 'Multi-sport', 'Other')
#print(train_labels_np[0:30])
#print(train_labels_np_bin[0:30])

In [9]:
#l_list = []
#for txt in text_np:
#    l_list.append( len(txt) )
    
#l_list[0:10]

#max(l_list)

In [10]:
maxlen = 207
#training_samples = 1491
#validation_samples = 500
max_words = 100000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_np)
sequences = tokenizer.texts_to_sequences(text_np)


In [11]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
text_input_train = pad_sequences(sequences, maxlen=maxlen)

print('Shape of data tensor:', text_input_train.shape)
print('Shape of label tensor:', train_labels_np.shape)


Found 1635 unique tokens.
Shape of data tensor: (1597, 207)
Shape of label tensor: (1597,)


In [12]:
label_encoder = LabelEncoder()

y_train_integer_encoded = label_encoder.fit_transform(train_labels_np)
#train_labels_npy_val_integer_encoded = label_encoder.fit_transform(y_val)

#y_train_integer_encoded
#y_train_integer_encodedtrain_labels_np
y_train_one_hot = to_categorical(y_train_integer_encoded, num_classes=7)
#y_val_one_hot = to_categorical(y_val_integer_encoded, num_classes=7)

In [13]:
#indices = np.arange(data.shape[0])
#np.random.shuffle(indices)
#data = data[indices]
#train_labels_np = train_labels_np[indices]
#x_train = data[:training_samples]
#y_train = train_labels_np[:training_samples]
#x_val = data[training_samples: training_samples + validation_samples]
#y_val = train_labels_np[training_samples: training_samples + validation_samples]

In [14]:
from keras.layers import Input, Embedding, Flatten, Dense, concatenate
from keras.models import Model

# Text input branch
text_input = Input(shape=(maxlen,), dtype='int32', name='text_input')
embedded_text = Embedding(max_words, 8)(text_input)
flattened_text = Flatten()(embedded_text)

# Tabular input branch
tabular_input = Input(shape=(tabular_features_scaled.shape[1],), name='tabular_input')
dense_tabular = Dense(16, activation='relu')(tabular_input)

# Combine branches
concatenated = concatenate([flattened_text, dense_tabular], axis=-1)
output = Dense(7, activation='sigmoid')(concatenated)

# Create and compile the model
model_combined = Model(inputs=[text_input, tabular_input], outputs=output)
model_combined.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_combined.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 207)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 207, 8)       800000      text_input[0][0]                 
__________________________________________________________________________________________________
tabular_input (InputLayer)      [(None, 36)]         0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 1656)         0           embedding[0][0]                  
______________________________________________________________________________________________

In [15]:
history_combined = model_combined.fit([text_input_train, tabular_features_scaled], y_train_one_hot,
epochs=80,
batch_size=32,
validation_split=.1)

Epoch 1/80
45/45 [==============================] - 1s 17ms/step - loss: 0.3788 - accuracy: 0.5261 - val_loss: 0.4801 - val_accuracy: 0.0000e+00
Epoch 2/80
45/45 [==============================] - 0s 10ms/step - loss: 0.3015 - accuracy: 0.5358 - val_loss: 0.4778 - val_accuracy: 0.0000e+00
Epoch 3/80
45/45 [==============================] - 0s 11ms/step - loss: 0.2793 - accuracy: 0.5518 - val_loss: 0.4585 - val_accuracy: 0.0000e+00
Epoch 4/80
45/45 [==============================] - 0s 10ms/step - loss: 0.2525 - accuracy: 0.6235 - val_loss: 0.4328 - val_accuracy: 0.0000e+00
Epoch 5/80
45/45 [==============================] - 0s 10ms/step - loss: 0.2261 - accuracy: 0.6778 - val_loss: 0.3889 - val_accuracy: 0.0938
Epoch 6/80
45/45 [==============================] - 0s 10ms/step - loss: 0.2038 - accuracy: 0.7154 - val_loss: 0.3626 - val_accuracy: 0.2875
Epoch 7/80
45/45 [==============================] - 0s 10ms/step - loss: 0.1860 - accuracy: 0.7550 - val_loss: 0.3256 - val_accuracy: 0.49

In [16]:
from keras.layers import Dropout

In [17]:
# Text input branch
text_input_deeper = Input(shape=(maxlen,), dtype='int32', name='text_input')
embedded_text_deeper = Embedding(max_words, 8)(text_input_deeper)
flattened_text_deeper = Flatten()(embedded_text_deeper)

# Tabular input branch
tabular_input_deeper = Input(shape=(tabular_features_scaled.shape[1],), name='tabular_input')
dense_tabular_p1 = Dense(64, activation='relu')(tabular_input_deeper)
dropout1 = Dropout(0.5)(dense_tabular_p1)  # Dropout for regularization
dense_tabular_deeper = Dense(32, activation='relu')(dropout1)

# Combine branches
concatenated_deeper = concatenate([flattened_text_deeper, dense_tabular_deeper], axis=-1)
output_deeper = Dense(7, activation='sigmoid')(concatenated_deeper)

# Create and compile the model
model_combined_deeper = Model(inputs=[text_input_deeper, tabular_input_deeper], outputs=output_deeper)
model_combined_deeper.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_combined_deeper.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tabular_input (InputLayer)      [(None, 36)]         0                                            
__________________________________________________________________________________________________
text_input (InputLayer)         [(None, 207)]        0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           2368        tabular_input[0][0]              
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 207, 8)       800000      text_input[0][0]                 
____________________________________________________________________________________________

In [18]:
history_combined_deeper = model_combined_deeper.fit([text_input_train, tabular_features_scaled], y_train_one_hot,
epochs=80,
batch_size=32,
validation_split=.1)

Epoch 1/80
45/45 [==============================] - 1s 14ms/step - loss: 0.3798 - accuracy: 0.5108 - val_loss: 0.4864 - val_accuracy: 0.0000e+00
Epoch 2/80
45/45 [==============================] - 0s 10ms/step - loss: 0.2901 - accuracy: 0.5407 - val_loss: 0.4861 - val_accuracy: 0.0000e+00
Epoch 3/80
45/45 [==============================] - 0s 11ms/step - loss: 0.2505 - accuracy: 0.6486 - val_loss: 0.4127 - val_accuracy: 0.0000e+00
Epoch 4/80
45/45 [==============================] - 0s 11ms/step - loss: 0.2178 - accuracy: 0.7112 - val_loss: 0.3587 - val_accuracy: 0.2688
Epoch 5/80
45/45 [==============================] - 0s 10ms/step - loss: 0.1966 - accuracy: 0.7404 - val_loss: 0.3419 - val_accuracy: 0.3313
Epoch 6/80
45/45 [==============================] - 0s 11ms/step - loss: 0.1818 - accuracy: 0.7537 - val_loss: 0.3219 - val_accuracy: 0.4500
Epoch 7/80
45/45 [==============================] - 0s 10ms/step - loss: 0.1695 - accuracy: 0.7731 - val_loss: 0.3200 - val_accuracy: 0.4375
E

In [19]:
model_combined.save_weights('../Model Objects/keras_out_new_varsv1.h5')
model_combined_deeper.save_weights('../Model Objects/keras_out_new_vars_deeperv1.h5')